In [15]:
from lively import  Solver,MoveShape,AddShape,Translation, Rotation,Transform,SmoothnessMacroObjective, CollisionAvoidanceObjective, JointLimitsObjective, BoxShape, CollisionSettingInfo, ScalarRange
from lxml import etree

# Define example BoxShape, AddShape, MoveShape, Delete, and CollisionSettingInfo 

box = BoxShape(name="Table",frame="world",physical=True,x=2,y=1,z=1.2,translation = Translation(x=0.0, y =0.0, z=0.0),rotation = Rotation(x=0.0,y=0.0,z=0.0,w=1.0))
box_1 = BoxShape(name="ToBeMovedOrDeleted",frame="world",physical=True,x=2,y=1,z=1.2,translation = Translation(x=0.0, y =0.0, z=1.0),rotation = Rotation(x=0.0,y=0.0,z=0.0,w=1.0))
addShape = AddShape(id = "box_1", shape = box_1)
moveShape = MoveShape(id = "box_1", translation = Translation(x=-1.0, y =0.0, z=4.0),rotation = Rotation(x=0.0,y=0.0,z=0.0,w=1.0))
deleteShape = "box_1" # Do not need to import classes from lively. Just pass in the string of the id of the shape you want to delete
collision = CollisionSettingInfo(d_max = 0.8, r = 0.0, a_max = 2.0, time_budget = 100, timed = False)

# Read the xml file into a string
xml_file = '../../tests/panda.xml'
tree = etree.parse(xml_file)
xml_string = etree.tostring(tree).decode()

#print(xml_string)
# Instantiate a new solver
solver = Solver(
  urdf=xml_string, # Full urdf as a string
  objectives={
      # An example objectives
      "smoothness":SmoothnessMacroObjective(name="MySmoothnessObjective",weight=5,joints=True,origin=False,links=True),
      "collision": CollisionAvoidanceObjective(name="MyCollisionAvoidanceObjective", weight=5),
      "jointLimit": JointLimitsObjective(name="MyJointLimitObjective", weight=5)
      
  },
  root_bounds=[
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0), # Translational, (x, y, z)
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0)  # Rotational, (r, p, y)
  ],
  shapes=[box], # Static environmental shape cannot be moved ot deleted. 
  collision_settings = collision

)

# Run solve to get a solved state
state = solver.solve(goals= {},weights = {},time = 0.0,shape_updates = [addShape,moveShape,deleteShape])
# Log the initial statse
print(state.origin.as_dicts())
print(state.joints)
print(state.frames["panda_link0"])
print(state.proximity)

AttributeError: 'builtins.BoxShape' object has no attribute 'clone'